In [ ]:
import pandas as pd
import numpy as np
import networkx as nx 


from os.path import join
import os

from dotenv import load_dotenv
load_dotenv()  

path = os.environ['PROJECT_PATH']

# Network Metric to Analyse 

- **Clustering statistics** - Number of addresses: “Each data point refers to the totals observed up to a given block, with weekly frequency, and time “flows” from the bottom left to the top right of the panel.” ([Campajola et al., 2022, p. 2](zotero://select/library/items/RYAYFRFR))
    
- **Degree distributions** - “The degree of a node represents the number of counterparts that entity exchanges tokens with, and is typically considered a measure of importance within the network. What is particularly relevant is to consider the distribution of degrees, as its shape is a direct consequence of the way in which the networks form.” ([Campajola et al., 2022, p. 3](zotero://select/library/items/RYAYFRFR)) ([pdf](zotero://open-pdf/library/items/LMUZEWY6?page=3))
    
- **Core-periphery structure** - “This *[edit. core-periphery structure]* is a macroscopic property of the network, that presents a split between a minority of nodes (the “core”) with a strong connectivity between themselves and the remaining nodes of the network (the “periphery”) that are mostly connected to core nodes and have relatively few links to other peripheral nodes.” ([Campajola et al., 2022, p. 4](zotero://select/library/items/RYAYFRFR)) ([pdf](zotero://open-pdf/library/items/LMUZEWY6?page=4))
    
    - “weekly transaction networks time-series and then consider the size of the core group as a fraction of the total size of the network” ([Campajola et al., 2022, p. 4](zotero://select/library/items/RYAYFRFR)) ([pdf](zotero://open-pdf/library/items/LMUZEWY6?page=4))
        
    - Relative size is used to assess centralisation or decentralisation trend
        
- **Mining concentration** - “The Nakamoto index provides us with a measure of the system’s distance from a 51% attack” ([Campajola et al., 2022, p. 5](zotero://select/library/items/RYAYFRFR)) ([pdf](zotero://open-pdf/library/items/LMUZEWY6?page=5))
    
- **Wealth inequality and spatial distribution -** “We define as wealth the balance held by an entity (address or cluster of addresses) at a given point in time, representing the funds that they are entitled to spend according to the blockchain ledger.” ([Campajola et al., 2022, p. 6](zotero://select/library/items/RYAYFRFR))
    
    - Token distribution and Gini Coefficient

In [ ]:
# load file

file = 'tx_all_uniq_addresses.edgelist'
G = nx.read_edgelist(join(path, file), data=False)

In [ ]:
G.number_of_nodes()


In [ ]:
G.number_of_edges()

# Statistically Validated Networks in Bipartite Complex Systems

For the analysis we sub-divide the our data into two sub-set.

- N_A = Set of 17 Token addresses 
    > |Note: Tokenmigration or wrapped version (e.g. wNXM) have been replace with the address of the corresponding address. 
- N_B = Remainder of 78700610 (78700627-17) addresses 
- k = 1 (always)
    >  From Tumminello et al., 2011, p. 3 - *Set A of the database is composed by 66 organisms (13 Archaea, 50 Bacteria and 3 unicellular Eukaryota) and set B by 4,873 COGs present in their genomes. The number of COGs in a genome is heterogeneous, ranging from 362 to 2,243. Similarly, COGs can be present in a different number of genomes. We call any COG that is present in k different genomes a k-COG. In the present system, k ranges between 3 and 66.*
    > By analogy Set A in our case tokens are homogenous sub-groups of 1 (different organism species)
- N_t < 1*17*(8)



In [ ]:
import pandas as pd
import numpy as np

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import itertools

from os.path import join

In [ ]:
# remove burner addresses 
known_burner_addresses = ['0x0000000000000000000000000000000000000000',
                        '0x0000000000000000000000000000000000000000',
                        '0x0000000000000000000000000000000000000001',
                        '0x0000000000000000000000000000000000000002',
                        '0x0000000000000000000000000000000000000003',
                        '0x0000000000000000000000000000000000000004',
                        '0x0000000000000000000000000000000000000005',
                        '0x0000000000000000000000000000000000000006',
                        '0x0000000000000000000000000000000000000007',
                        '0x000000000000000000000000000000000000dead']

In [ ]:
# read snapshot
ddf = dd.read_csv(join(path, 'token_balance_lookup_tables/token_holder_snapshot_balance_14967365.csv'))

# we only consider non-zero at snapshot 
ddf = ddf[ddf.value > 0]

#  filter latest tokens 
df_addresses = pd.read_csv('assets/df_final_token_selection_20221209.csv')
ddf = ddf[ddf.token_address.isin(df_addresses.address) == True]

# remove known burner addresses 
ddf = ddf[ddf.address.isin(known_burner_addresses) == False]


In [ ]:
from scipy.stats import hypergeom 
# hypergeom.cdf(x,M,n,N,loc=0)	

import pandas as pd
def main(address1, address2, pop_size):

    # get unique addresses 
    token1_uniqa = ddf[ddf.token_address == address1].address.unique()
    token2_uniqa = ddf[ddf.token_address == address2].address.unique()

    # calcluate intersection 
    token1_token2_uniqa_intersection = np.intersect1d(token1_uniqa,token2_uniqa, assume_unique=True)

    # calcualte number 
    len_token1 = len(token1_uniqa)
    len_token2 = len(token2_uniqa)
    len_intersection = len(token1_token2_uniqa_intersection)

    # calculate hyptoge

    # Define the parameters of the distribution
    M = pop_size  # population size
    n = len_token1  # number of draws
    K = len_token2  # number of successes in population
    x = len_intersection    # number of successes in draws

    # Compute the cumulative probability of obtaining at most x successes
    pvalue = 1 - hypergeom.cdf(x, M, n, K)
    
    print(f'token_address {address1} has {len_token1} Unique Addresses | token_address {address2} has {len_token2} Unique Addresses | Intersection: {len_intersection}, p value: {pvalue}')

    return pvalue

In [ ]:
# # Demo
# # test tokens 
# test_token1 = '0x111111111117dc0aa78b770fa6a738034120c302'
# test_token2 = '0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e'

# # calc pop_size
# pop_size = len(ddf.address.unique()) 

# # main
# main(test_token1,test_token2, pop_size)

In [ ]:
pop_size = len(ddf.address.unique()) 
p_dict = {}

for combination in itertools.combinations(df_addresses.address, 2): 

    pvalue = main(combination[0],combination[1], pop_size)

    p_dict[combination] = pvalue

### Evaluate p values 



In [ ]:
df_pvalues = pd.DataFrame.from_dict(p_dict, orient='index')

df_pvalues.reset_index(inplace=True)

df_pvalues.columns =  ['combination', 'p_value']



In [ ]:
from statsmodels.stats.multitest import multipletests as m_tests
# Ref.: https://www.statsmodels.org/devel/generated/statsmodels.stats.multitest.multipletests.html#statsmodels.stats.multitest.multipletests
m_test = m_tests(pvals=df_pvalues.p_value, alpha=0.01, method='bonferroni')


In [ ]:
df_pvalues['m_test_result'] = m_test[0]
df_pvalues['m_test_value'] = m_test[1]
df_pvalues.to_csv('pvalues_snapshot_14967365.csv')

### Visualise

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from ast import literal_eval


df_pvalues = pd.read_csv('pvalues_snapshot_14967365.csv')

df_pvalues_validated = df_pvalues[df_pvalues.m_test_result == True]
df_pvalues_validated.combination = df_pvalues_validated.combination.apply(lambda x: literal_eval(x))  
# Create an empty graph
G = nx.Graph()

# Add the edges to the graph
G.add_edges_from(df_pvalues_validated.combination)

# create labels 
df_a_fil = df_addresses[df_addresses.address.isin(list(G.nodes()))]
labels = df_a_fil[['address', 'name']].set_index('address').to_dict()['name']

# visualise netwotk 
nx.draw(G, labels=labels)

#show
plt.show()
plt.savefig('vNetwork_14967365.png')


### Quantify basic network metrics 

- The number of nodes and edges in the graph. This can give you a sense of the size and complexity of the graph.

In [ ]:
G.number_of_edges()
G.number_of_nodes()

- The degree distribution of the graph. This describes the number of connections (or "degrees") that each node has.

In [ ]:
## degree 
degrees = dict(G.degree())
# Calculate the average degree of the graph
avg_degree = sum(degrees.values()) / len(degrees)
print(avg_degree)


# Calculate the maximum degree of the graph
max_degree = max(degrees.values())
print(max_degree)

# Calculate the minimum degree of the graph
min_degree = min(degrees.values())
print(min_degree)




- The diameter and average shortest path length of the graph. The diameter is the longest shortest path between any two nodes in the graph, and the average shortest path length is the average of all the shortest paths in the graph. These statistics can give you a sense of how "connected" the graph is.

In [ ]:
# diameter
nx.diameter(G)

# average shortest path
nx.average_shortest_path_length(G)


- The density of the graph. This is the ratio of the number of edges in the graph to the maximum number of edges that the graph could have. A dense graph has a high number of edges, while a sparse graph has a low number of edges.

In [ ]:
nx.density(G)

- The degree centrality and betweenness centrality of the nodes in the graph. The degree centrality of a node is a measure of how many connections it has, and the betweenness centrality of a node is a measure of how important it is for connecting other nodes in the graph

In [ ]:
# centrality 
g_degree_centrality = dict(nx.degree_centrality(G))
sum(g_degree_centrality.values())/ len(g_degree_centrality)


# betweeness 
g_betweeness_centrality = dict(nx.betweenness_centrality(G))
sum(g_betweeness_centrality.values())/ len(g_betweeness_centrality)



- clustering coefficient - Algorithms to characterize the number of triangles in a graph.

In [ ]:
g_triangles = dict(nx.triangles(G))
sum(g_triangles.values())/ len(g_triangles)

g_clustering = dict(nx.clustering(G))
sum(g_clustering.values())/ len(g_clustering)



### repeat for other snapshot 

In [ ]:
# imports

import pandas as pd
import numpy as np

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import itertools

from os.path import join
from scipy.stats import hypergeom 
from statsmodels.stats.multitest import multipletests as m_tests
import matplotlib.pyplot as plt
from ast import literal_eval
import networkx as nx


summary_stats = {}


progress = 0 

# snapshot selection 
df_snapshot = pd.read_csv('assets/snapshot_selection.csv')

# address selection 
df_addresses = pd.read_csv('assets/df_final_token_selection_20221209.csv')

# burner addresses 
# remove burner addresses 
known_burner_addresses = ['0x0000000000000000000000000000000000000000',
                        '0x0000000000000000000000000000000000000000',
                        '0x0000000000000000000000000000000000000001',
                        '0x0000000000000000000000000000000000000002',
                        '0x0000000000000000000000000000000000000003',
                        '0x0000000000000000000000000000000000000004',
                        '0x0000000000000000000000000000000000000005',
                        '0x0000000000000000000000000000000000000006',
                        '0x0000000000000000000000000000000000000007',
                        '0x000000000000000000000000000000000000dead']



#### functions ####

def main(address1, address2, pop_size):

    # get unique addresses 
    token1_uniqa = ddf[ddf.token_address == address1].address.unique()
    token2_uniqa = ddf[ddf.token_address == address2].address.unique()

    # calcluate intersection 
    token1_token2_uniqa_intersection = np.intersect1d(token1_uniqa,token2_uniqa, assume_unique=True)

    # calcualte number 
    len_token1 = len(token1_uniqa)
    len_token2 = len(token2_uniqa)
    len_intersection = len(token1_token2_uniqa_intersection)

    # calculate hyptoge

    # Define the parameters of the distribution
    M = pop_size  # population size
    n = len_token1  # number of draws
    K = len_token2  # number of successes in population
    x = len_intersection    # number of successes in draws

    # Compute the cumulative probability of obtaining at most x successes
    pvalue = 1 - hypergeom.cdf(x, M, n, K)
    
    # print(f'token_address {address1} has {len_token1} Unique Addresses | token_address {address2} has {len_token2} Unique Addresses | Intersection: {len_intersection} | p value: {pvalue}')

    return pvalue

###################


for snapshot in df_snapshot['Block Height']:

    # Info
    items_left = len(df_snapshot) - progress
    progress += 1
    print(
        f"Current Snapshot: {snapshot} || Items processed: {progress} || Items left: { (items_left) }"
    )

    ## formating of data
    # load data
    ddf = dd.read_csv(join(path, f'token_balance_lookup_tables/token_holder_snapshot_balance_{snapshot}.csv'))

    # filter data 
    ddf = ddf[ddf.value > 0]
    ddf = ddf[ddf.token_address.isin(df_addresses.address) == True]

    # remove known burner addresses 
    ddf = ddf[ddf.address.isin(known_burner_addresses) == False]

    # population size
    pop_size = len(ddf.address.unique()) 
    p_dict = {}

    # reduce address list 
    
    present_addresses = list(ddf.token_address.unique().compute())

    if len(present_addresses) == 1:

        print('One address only')
        stats = {'nodes': np.nan, 'possible_nodes': len(present_addresses), 'edges': np.nan, 'avg_degree_path': np.nan, 'min_degree_path': np.nan, 'max_degree_path': np.nan,'diameter': np.nan, 'avg_shortest_path': np.nan, 'density': np.nan, 'degree_centrality_avg': np.nan, 'degree_centrality_min': np.nan, 'degree_centrality_max': np.nan, 'betweeness_centrality_avg': np.nan, 'betweeness_centrality_min': np.nan, 'betweeness_centrality_max': np.nan, 'triangles_avg': np.nan, 'triangles_min': np.nan, 'triangles_max': np.nan}


    else:

        try:
            # iterations 
            for combination in itertools.combinations(present_addresses, 2): 

                pvalue = main(combination[0],combination[1], pop_size)

                p_dict[combination] = pvalue

            ## Evaluate pvalues
            # store pvalues 
            df_pvalues = pd.DataFrame.from_dict(p_dict, orient='index')
            df_pvalues.reset_index(inplace=True)
            df_pvalues.columns =  ['combination', 'p_value']

            # value test 
            m_test = m_tests(pvals=df_pvalues.p_value, alpha=0.01, method='bonferroni')
            df_pvalues['m_test_result'] = m_test[0]
            df_pvalues['m_test_value'] = m_test[1]
            df_pvalues.to_csv(join(path, f'output/pvalues_{snapshot}.csv'))


            ## Build graph
            # filter df  
            df_pvalues_validated = df_pvalues[df_pvalues.m_test_result == True]

            # Create an empty graph
            G = nx.Graph()

            # Add the edges to the graph
            G.add_edges_from(df_pvalues_validated.combination)

            # create labels 
            df_a_fil = df_addresses[df_addresses.address.isin(list(G.nodes()))]
            labels = df_a_fil[['address', 'name']].set_index('address').to_dict()['name']

            # visualise netwotk 
            nx.draw(G, labels=labels)

            #show
            plt.savefig(join(path, f'output/pics/pic_vNetwork_{snapshot}.png'))

            # clear img
            plt.clf() 

            ## descriptve statistic
            g_nodes = G.number_of_nodes()
            g_edges = G.number_of_edges()
            ## degree 
            g_degrees = dict(G.degree())
            # Calculate the average degree of the graph

            try: 
                g_avg_degree = sum(g_degrees.values()) / len(g_degrees)
                g_max_degree = max(g_degrees.values())
                g_min_degree = min(g_degrees.values())
            except: 
                g_avg_degree = np.nan
                g_max_degree = np.nan
                g_min_degree = np.nan


            # diameter
            g_diameter = nx.diameter(G)

            # average shortest path
            g_avg_shortest_path = nx.average_shortest_path_length(G)


            g_density = nx.density(G)

            # centrality 
            g_degree_centrality = dict(nx.degree_centrality(G))

            try: 
                g_degree_centrality_avg = sum(g_degree_centrality.values())/ len(g_degree_centrality)
                g_degree_centrality_min = min(g_degree_centrality.values())
                g_degree_centrality_max = max(g_degree_centrality.values())

            except:

                g_degree_centrality_avg = np.nan
                g_degree_centrality_min = np.nan
                g_degree_centrality_max = np.nan

            # betweeness 
            g_betweeness_centrality = dict(nx.betweenness_centrality(G))

            try: 
                g_betweeness_centrality_avg = sum(g_betweeness_centrality.values())/ len(g_betweeness_centrality)
                g_betweeness_centrality_min = min(g_betweeness_centrality.values())
                g_betweeness_centrality_max = max(g_betweeness_centrality.values())

            except:
                g_betweeness_centrality_avg = np.nan
                g_betweeness_centrality_min = np.nan
                g_betweeness_centrality_max = np.nan

            # triangles 
            g_triangles = dict(nx.triangles(G))

            try: 
                g_triangles_avg = sum(g_triangles.values())/ len(g_triangles)
                g_triangles_min = min(g_triangles.values())
                g_triangles_max = max(g_triangles.values())
            except: 
                g_triangles_avg = np.nan
                g_triangles_min = np.nan
                g_triangles_max = np.nan

            # g_clustering = dict(nx.clustering(G))
            # g_clustering_avg = sum(g_clustering.values())/ len(g_clustering)
            # g_clustering_min = min(g_clustering.values())
            # g_clustering_max = max(g_clustering.values())

            stats = {'nodes': g_nodes, 'possible_nodes': len(present_addresses), 'edges': g_edges, 'avg_degree_path': g_avg_degree, 'min_degree_path': g_min_degree, 'max_degree_path': g_max_degree,'diameter': g_diameter, 'avg_shortest_path': g_avg_shortest_path, 'density': g_density, 'degree_centrality_avg': g_degree_centrality_avg, 'degree_centrality_min': g_degree_centrality_min, 'degree_centrality_max': g_degree_centrality_max, 'betweeness_centrality_avg': g_betweeness_centrality_avg, 'betweeness_centrality_min': g_betweeness_centrality_min, 'betweeness_centrality_max': g_betweeness_centrality_max, 'triangles_avg': g_triangles_avg, 'triangles_min': g_triangles_min, 'triangles_max': g_triangles_max}
            # 'clustering_avg': g_clustering_avg, 'clustering_min': g_clustering_min, 'clustering_max': g_clustering_max }

        except: 
            stats = {'nodes': np.nan, 'possible_nodes': len(present_addresses), 'edges': np.nan, 'avg_degree_path': np.nan, 'min_degree_path': np.nan, 'max_degree_path': np.nan,'diameter': np.nan, 'avg_shortest_path': np.nan, 'density': np.nan, 'degree_centrality_avg': np.nan, 'degree_centrality_min': np.nan, 'degree_centrality_max': np.nan, 'betweeness_centrality_avg': np.nan, 'betweeness_centrality_min': np.nan, 'betweeness_centrality_max': np.nan, 'triangles_avg': np.nan, 'triangles_min': np.nan, 'triangles_max': np.nan}
    

summary_stats[snapshot] = stats




In [ ]:


## descriptve statistic
g_nodes = G.number_of_nodes()
g_edges = G.number_of_edges()
## degree 
g_degrees = dict(G.degree())
# Calculate the average degree of the graph
g_avg_degree = sum(g_degrees.values()) / len(g_degrees)
g_max_degree = max(g_degrees.values())
g_min_degree = min(g_degrees.values())


# diameter
g_diameter = nx.diameter(G)

# average shortest path
g_avg_shortest_path = nx.average_shortest_path_length(G)


g_density = nx.density(G)

# centrality 
g_degree_centrality = dict(nx.degree_centrality(G))
g_degree_centrality_avg = sum(g_degree_centrality.values())/ len(g_degree_centrality)
g_degree_centrality_min = min(g_degree_centrality.values())
g_degree_centrality_max = max(g_degree_centrality.values())

# betweeness 
g_betweeness_centrality = dict(nx.betweenness_centrality(G))
g_betweeness_centrality_avg = sum(g_betweeness_centrality.values())/ len(g_betweeness_centrality)
g_betweeness_centrality_min = min(g_betweeness_centrality.values())
g_betweeness_centrality_max = max(g_betweeness_centrality.values())

# triangles 
g_triangles = dict(nx.triangles(G))
g_triangles_avg = sum(g_triangles.values())/ len(g_triangles)
g_triangles_min = min(g_triangles.values())
g_triangles_max = max(g_triangles.values())

g_clustering = dict(nx.clustering(G))
g_clustering_avg = sum(g_clustering.values())/ len(g_clustering)
g_clustering_min = min(g_clustering.values())
g_clustering_max = max(g_clustering.values())

stats = {'nodes': g_nodes, 'edges': g_edges, 
'avg_degree_path': g_avg_degree, 'min_degree_path': g_min_degree, 'max_degree_path': g_max_degree,
'diameter': g_diameter,
'avg_shortest_path': g_avg_shortest_path, 'density': g_density,
'degree_centrality_avg': g_degree_centrality_avg, 'degree_centrality_min': g_degree_centrality_min, 'degree_centrality_max': g_degree_centrality_max, 
'betweeness_centrality_avg': g_betweeness_centrality_avg, 'betweeness_centrality_min': g_betweeness_centrality_min, 'betweeness_centrality_max': g_betweeness_centrality_max,
'triangles_avg': g_triangles_avg, 'triangles_min': g_triangles_min, 'triangles_max': g_triangles_max,
'clustering_avg': g_clustering_avg, 'clustering_min': g_clustering_min, 'clustering_max': g_clustering_max }

In [ ]:
test= {}

test[123] = stats
test[432] = stats 